# LCEL(LangChain Expression Language)
## 기본 예시: 프롬프트 + 모델 + 출력 파서
가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 사용하는 것

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

def langsmith(project_name=None, set_enable=True):

    if set_enable:
        result = os.environ.get("LANGCHAIN_API_KEY")
        if result is None or result.strip() == "":
            print(
                "LangChain API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954"
            )
            return
        os.environ["LANGCHAIN_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")


langsmith("LANGCHAIN-STUDY")

LangSmith 추적을 시작합니다.
[프로젝트명]
LANGCHAIN-STUDY


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [4]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [11]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)
LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합

```
chain = prompt | model | output_parser
```

`|` 기호는 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

`prompt -> model -> output_parser`

In [12]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = AzureChatOpenAI()

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [13]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [14]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Using legacy API key. Please generate a new API key."}')trace=485f250a-243c-46a5-913c-a75492aacffd,id=485f250a-243c-46a5-913c-a75492aacffd; trace=485f250a-243c-46a5-913c-a75492aacffd,id=aed9df12-7d6b-4bba-b9d8-027a110cdbad; trace=485f250a-243c-46a5-913c-a75492aacffd,id=1b14d69c-988a-4112-a9fb-21a8feb7a0ff


AIMessage(content='인공지능 모델의 학습 원리를 쉽게 설명하자면, "경험을 통해 배운다"라고 할 수 있습니다. 여기서 경험은 데이터, 즉 입력된 정보를 의미합니다. 다음은 인공지능 모델이 어떻게 학습하는지에 대한 간단한 단계입니다.\n\n1. **데이터 수집**: 먼저, 모델이 학습할 수 있는 데이터가 필요합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집된 데이터는 보통 혼란스러울 수 있기 때문에, 모델이 이해할 수 있도록 정리하고 가공합니다. 예를 들어, 이미지를 크기 조정하거나 텍스트를 정제하는 과정이 여기에 포함됩니다.\n\n3. **모델 구조 생성**: 인공지능 모델은 여러 층(layer)으로 구성된 신경망(neural network)이라는 구조로 이루어져 있습니다. 각 층은 입력된 정보를 처리하고 다음 층으로 전달하는 역할을 합니다.\n\n4. **학습 과정**: 모델은 주어진 데이터에 대해 예측을 시도하고, 그 예측이 실제 결과와 얼마나 차이가 나는지를 평가합니다. 이 과정에서 \'손실 함수(loss function)\'라는 것을 사용하여 예측의 정확도를 측정합니다.\n\n5. **오류 수정**: 모델이 예측한 결과가 틀렸다면, 그 오류를 기반으로 모델의 내부 파라미터(가중치)를 조정합니다. 이 과정을 \'역전파(backpropagation)\'라고 부릅니다.\n\n6. **반복 학습**: 위의 과정(예측, 오류 평가, 파라미터 조정)을 여러 번 반복하면서 모델은 점점 더 정확한 예측을 할 수 있게 됩니다. 많은 데이터를 통해 학습할수록 성능이 향상됩니다.\n\n7. **검증 및 테스트**: 학습이 끝난 후에는 모델의 성능을 검증하기 위해 새로운 데이터로 테스트합니다. 이 데이터를 통해 모델이 실제 상황에서도 잘 작동하는지를 확인합니다.\n\n이러한 과정을 통해 인공지능 모델은 특정 작업을 수행할 수 있는 능력을 가지게 됩니다. 결과적으로, 데이터와 경험을 통해 스스로 

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Using legacy API key. Please generate a new API key."}')trace=485f250a-243c-46a5-913c-a75492aacffd,id=485f250a-243c-46a5-913c-a75492aacffd; trace=485f250a-243c-46a5-913c-a75492aacffd,id=1b14d69c-988a-4112-a9fb-21a8feb7a0ff
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Using legacy API key. Please generate a new API key."}')trace=e32c523d-354f-4202-918c-ed26b280a666,id=e32c523d-354f-4202-918c-ed26b280a666; trace=e32c523d-354f-4202-918c-ed26b280a666,id=01f731b6-5f34-4afb-b5cd-465a77f0e9df; trace=e32c523d-354f-4202-918c-ed26b280a666,

In [15]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 주로 '데이터'와 '학습 알고리즘'을 기반으로 합니다. 간단히 설명하자면 다음과 같습니다.

1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 양의 데이터를 필요로 합니다. 이 데이터는 텍스트, 이미지, 소리 등 다양한 형태일 수 있습니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진을 많이 모아야 합니다.

2. **훈련(학습)**: 모델은 데이터를 통해 패턴을 배우게 됩니다. 이 단계에서는 모델이 입력(예: 사진)과 출력(예: 고양이 또는 개)을 연결하는 방법을 익히게 됩니다. 이를 위해 수학적 알고리즘이 사용되며, 주로 신경망(neural network)이라는 구조를 활용합니다.

3. **손실 함수**: 모델이 예측한 결과와 실제 정답 간의 차이를 계산하는 방법입니다. 이 차이를 줄이기 위해 모델의 파라미터(가중치)를 조정합니다. 손실 함수는 모델이 얼마나 잘 학습하고 있는지를 평가하는 기준이 됩니다.

4. **최적화**: 손실 함수를 최소화하기 위해 모델의 파라미터를 조정하는 과정입니다. 이것은 반복적으로 이루어지며, 모델이 점점 더 정확한 예측을 하도록 돕습니다. 보통 '경사 하강법'이라는 방법이 사용됩니다.

5. **검증 및 평가**: 학습이 끝난 후, 모델이 새로운 데이터에 대해 얼마나 잘 작동하는지를 평가합니다. 이를 통해 모델의 성능을 확인하고, 필요시 조정할 수 있습니다.

6. **실행**: 마지막으로, 학습된 모델은 실제 환경에서 사용됩니다. 예를 들어, 사용자가 고양이 사진을 올리면, 모델이 이를 분석하여 고양이인지 개인지 판단합니다.

이러한 과정을 통해 인공지능 모델은 경험을 쌓고, 점점 더 똑똑해져 가는 것입니다.

In [16]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [17]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [18]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"인공지능 모델의 학습 원리를 쉽게 설명하자면, 마치 사람의 학습 과정과 비슷하다고 볼 수 있습니다. 다음은 그 과정의 기본적인 단계입니다.\n\n1. **데이터 수집**: 인공지능 모델은 많은 양의 데이터가 필요합니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진을 많이 모아야 합니다.\n\n2. **데이터 전처리**: 수집한 데이터는 보통 원래 형태로는 사용할 수 없기 때문에, 정리하고 가공하는 과정이 필요합니다. 사진의 크기를 맞추거나, 필요 없는 정보를 제거하는 등의 작업이 이루어집니다.\n\n3. **모델 선택**: 다양한 인공지능 알고리즘 중에서 문제에 적합한 모델을 선택합니다. 예를 들어, 이미지 인식에는 CNN(합성곱 신경망)이라는 모델이 자주 사용됩니다.\n\n4. **학습 과정**: 모델은 주어진 데이터를 통해 학습을 시작합니다. 이 과정은 '훈련'이라고도 부르며, 모델이 입력 데이터를 보고 정답(레이블)과 비교하여 얼마나 잘 맞추는지를 평가합니다. 이를 통해 모델의 내부 파라미터(가중치)를 조정하면서 점점 더 정확하게 예측할 수 있도록 합니다.\n\n5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(검증 데이터)를 사용합니다. 이 데이터를 통해 모델이 실제 상황에서도 잘 작동하는지를 확인합니다.\n\n6. **예측**: 학습이 완료되고 성능이 검증된 모델은 실제 데이터에 대해 예측을 할 수 있습니다. 예를 들어, 새로운 사진이 들어왔을 때 그것이 고양이인지 개인지 판단할 수 있습니다.\n\n7. **지속적인 개선**: 모델은 새로운 데이터가 들어오거나 시간이 지나면서 지속적으로 업데이트되고 개선될 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 데이터를 이해하고, 패턴을 학습하여 문제를 해결하는 능력을 키워나갑니다."

In [19]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 과정을 거칩니다:

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해 많은 데이터를 모읍니다. 예를 들어, 고양이와 개의 사진을 모아서 모델이 이 두 동물을 구분할 수 있도록 준비합니다.

2. **데이터 전처리**: 수집한 데이터를 모델이 이해할 수 있는 형태로 변환합니다. 예를 들어, 이미지의 크기를 조정하거나, 색상을 표준화하는 등의 작업을 합니다.

3. **모델 선택**: 문제에 맞는 적절한 인공지능 모델을 선택합니다. 예를 들어, 이미지 분류를 위해 신경망(Neural Network) 모델을 사용할 수 있습니다.

4. **학습**: 모델이 데이터를 통해 패턴을 인식하도록 훈련합니다. 이 과정에서는 데이터의 입력과 정답(예: 고양이 또는 개)을 사용하여 모델의 내부 파라미터를 조정합니다. 이때, 오차를 줄이기 위해 경량화된 알고리즘인 '역전파(Backpropagation)'를 사용합니다.

5. **검증**: 학습이 끝난 모델을 새로운 데이터로 테스트하여 얼마나 잘 학습했는지를 평가합니다. 이 단계에서 모델의 성능을 확인할 수 있습니다.

6. **조정**: 모델의 성능이 만족스럽지 않으면, 데이터 양을 늘리거나, 모델의 구조를 변경하거나, 하이퍼파라미터(학습률 등)를 조정하여 다시 학습합니다.

7. **배포**: 모델이 충분히 학습되면, 실제 응용 프로그램에 배포하여 사용합니다. 예를 들어, 사진을 업로드하면 고양이인지 개인지 분류해 주는 앱으로 활용할 수 있습니다.

이런 과정을 통해 인공지능 모델은 데이터를 학습하고, 새로운 데이터를 처리할 수 있는 능력을 갖추게 됩니다.

### 템플릿을 변경하여 적용

In [20]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = AzureChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [21]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [22]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
  **Waiter:** Good evening! Welcome to our restaurant. How many people are in your party?  
  **You:** Just one, please.  
  **Waiter:** Great! Here’s the menu. Can I get you something to drink while you look?  
  **You:** Yes, I’d like a glass of water, please.  
  **Waiter:** Sure! Take your time. Let me know when you’re ready to order.  
  **You:** Thank you! I’ll have the grilled chicken salad, please.  
  **Waiter:** Excellent choice! Would you like any dressing with that?  
  **You:** Yes, please. I’ll have the vinaigrette.  
  **Waiter:** Perfect! I’ll get that order in for you.  

- 한글 해석:  
  **웨이터:** 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이신가요?  
  **당신:** 혼자입니다, 부탁드립니다.  
  **웨이터:** 좋습니다! 메뉴를 드리겠습니다. 보시는 동안 음료수는 필요하신가요?  
  **당신:** 네, 물 한 잔 주세요.  
  **웨이터:** 알겠습니다! 천천히 보세요. 주문 준비되시면 말씀해 주세요.  
  **당신:** 감사합니다! 저는 그릴에 구운 치킨 샐러드로 할게요.  
  **웨이터:** 훌륭한 선택입니다! 드레싱은 어떤 걸 원하시나요?  
  **당신:** 네, 비네그레트로 주세요.  
  **웨이터:** 완벽합니다! 주문해 드리겠습니다.


In [23]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
  **Waiter:** Good evening! Welcome to our restaurant. How can I help you today?  
  **You:** Hi! I'd like to see the menu, please.  
  **Waiter:** Of course! Here you go. Do you have any questions about the menu?  
  **You:** Yes, what do you recommend?  
  **Waiter:** Our grilled salmon is very popular, and the pasta primavera is also a favorite.  
  **You:** That sounds great! I’ll have the grilled salmon, please.  
  **Waiter:** Would you like any sides with that?  
  **You:** Yes, I’d like a side salad, please.  
  **Waiter:** Perfect! Anything to drink?  
  **You:** I’ll have a glass of water, please.  
  **Waiter:** Great choice! I’ll put that order in for you.  

- 한글 해석:  
  **웨이터:** 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 오늘 무엇을 도와드릴까요?  
  **당신:** 안녕하세요! 메뉴를 보고 싶어요.  
  **웨이터:** 물론이죠! 여기 있습니다. 메뉴에 대해 질문이 있으신가요?  
  **당신:** 네, 추천해 주실 만한 것이 있나요?  
  **웨이터:** 저희 그릴에 구운 연어가 아주 인기가 많고, 파스타 프리마베라도 인기 있습니다.  
  **당신:** 좋네요! 그릴에 구운 연어로 주문할게요.  
  **웨이터:** 사이드 메뉴는 무엇을 원하시나요?  
  **당

In [24]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hi there! I’d like to order a pizza, please.
B: Sure! What size would you like?
A: I’ll have a large, please. Can I get half pepperoni and half veggie?
B: Absolutely! Would you like any extra toppings?
A: Yes, add some olives on the veggie side.
B: Great choice! Would you like any drinks with that?
A: Yes, I’ll have a 2-liter soda, please.
B: Perfect! Your total comes to $25. Would you like that for delivery or pick-up?
A: I’ll pick it up, thank you. How long will it take?
B: It should be ready in about 20 minutes.
A: Sounds good! Thank you!

- 한글 해석:
A: 안녕하세요! 피자를 주문하고 싶어요.
B: 물론이죠! 어떤 사이즈로 하시겠어요?
A: 큰 사이즈로 주세요. 반은 페퍼로니, 반은 채소로 해주세요.
B: 알겠습니다! 추가 토핑이 필요하신가요?
A: 네, 채소 쪽에 올리브를 추가해 주세요.
B: 좋은 선택이에요! 음료는 필요하신가요?
A: 네, 2리터 탄산음료 하나 주세요.
B: 완벽해요! 총 금액은 25달러입니다. 배달할까요, 아니면 픽업할까요?
A: 픽업할게요, 감사합니다. 얼마나 걸릴까요?
B: 대략 20분 후에 준비될 거예요.
A: 좋네요! 감사합니다!